<a href="https://colab.research.google.com/github/arashkol/python_class/blob/main/wikipedia_click.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Wikipedia artcile recommender
##This program receives a keyword and recommends Wikipedia artciles in order to better understand the keyword. This recommender system uses click behavior of the users and the search to find related artciles.

The idea is that if the users have frequently clicked on a link in a wikipedia pages, while reading that page, the target/clicked page is a prerequisite for the current page. In case no click data is available, relevant artciles to the keyword as a result of the Wikipedia fuzzy search will be recommended to the user.

In this program Wikipedia API are used for search and retrival of information. The monthly published click stream of Wikipedia is also used as a guide for the recommender. The clisk stream could be downlowded here:


https://dumps.wikimedia.org/other/clickstream

##Technologies used:
[for loop](#for_loop)

[while loop](#while_loop)

[class](#class)

[class inheritance](#class_inheritance)

[function](#function)

[recursive function](#recursive_function)


In [4]:
#Installing the wikipedia module for using Wikipedia API
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=3a4b23f6ce5d142574f2eb6fdaac76afdeb8170619c12e334be1798e9c542513
  Stored in directory: /root/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


#Importing libraries
##main libraries used: 
###**Pandas** for data processing
###**Wikipedia** for retrieving information about Wikipedia articles using Wikipedia API

In [5]:
from curses.ascii import isascii #recognises if the letters a latin and numerical
import requests #used for web interaction
import gzip #used for unzipping the downloaded wikipedia click stream
import re #reqular expressions for string processing
from urllib.parse import quote #for replacing special charachters inside URLs with %xx escape.
import pandas as pd #used for tabular data anaylisis 
import wikipedia # Wikipedia API module
import random as rnd #for random integer generation
import numpy as np #for arithmatic operations

##Caiustion: if the data is already preprocessed and saved, ignore the following rows and start from: [Load preprocced data.](#cell-id)


##Retrieving data from the Wikipedia click-stream:
###Here the click data of several months is retrieved and concatinated.

Techniques used:
**for-loop**

In [6]:
# Define a list of month names to retrieve
#One month of clicks may not cover all keywords we need
months = ["2021-07", "2021-08", "2021-09", "2021-10", "2021-11", "2021-12"]

# Initialize an empty list to store the data
all_data = []

# Loop through each month and retrieve the data
for month in months:
    url = f"https://dumps.wikimedia.org/other/clickstream/{month}/clickstream-zhwiki-{month}.tsv.gz"
    response = requests.get(url)
    data = gzip.decompress(response.content)
    
    # Append the data to the running list
    all_data.append(data)

# Concatenate all data together into a single string
all_data = b"".join(all_data)

##Data cleaning and preparation

In [7]:
#convert bytes to unicode 
data = all_data.decode("utf-8")
#we have to do this because the all_data typy is byte encoded to "utf-8"

In [8]:
#An exampe of how utf-8 encoding works:
array = "München Ü"
byte_array = array.encode("utf-8")
char_to_find = "ü".encode("utf-8")  # encoded representation of "ü" in utf-8
index_of_char = byte_array.index(char_to_find)
print(array)
print(char_to_find)
print(byte_array)
print(index_of_char)


München Ü
b'\xc3\xbc'
b'M\xc3\xbcnchen \xc3\x9c'
1


In [9]:
# Check if the  string contains illegal characters: -~ -ÿĀ-῿Ⰰ-퟿豈-﷏ﷰ-�
def good_string(string_):
    if re.search(r"[^\u0020-\u007E\u00A0-\u00FF\u0100-\u1FFF\u2C00-\uD7FF\uF900-\uFDCF\uFDF0-\uFFFD]", string_):
        return False

    if (string_ == None): return False
    # Check if the string is properly encodable in UTF-8
    try:
        string_.encode("utf-8")
    except UnicodeEncodeError:
        return False

    return True

In [10]:
#check if the string is actually a number
def is_number(string_):
    try:
        float(string_)
        return True
    except ValueError:
        return False

In [11]:
#does the string contain just ASCII charachters? For example, Ü,Ö,Ä are not ASCII
def has_only_ascii(string_):
    return all(char.isascii() for char in string_)

In [12]:
#Example of ASCII vs non-ASCII characters recognized inside strings by has_only_ascii() function
print(has_only_ascii("Munich"), has_only_ascii("München"), has_only_ascii("Мюншен"), has_only_ascii("مونیخ")  )

True False False False


In [13]:
#define an empty dataset/DataFrame
df_clicks = pd.DataFrame(columns=['from','clicks','to']) 

##Data preprocessing
<a name="for_loop"></a>

In [ ]:
for line in data.split("\n"):

    columns = line.split("\t")

    # Skip the header row and the row containing non-keyword strings
    if columns[0] == "source" or len(columns)<2 \
    or columns[0] == "other-empty" \
    or "other-search" in columns[0] or "other-internal" in columns[0] or "other-external" in columns[0]\
    or "other-other" in columns[0] or "other-other" in columns[2]\
    or "other-empty" in columns[1]\
    or "other-search" in columns[1] or "other-internal" in columns[1] or "other-external" in columns[1]\
    or "other-empty" in columns[2]\
    or "other-search" in columns[2] or  "other-internal" in columns[2] or "other-external" in columns[2]\
    or not is_number(columns[3]) or not has_only_ascii(columns[0])or not has_only_ascii(columns[1]):
        continue

    #replace special characters in URLs with xx% charachters e.g.: "+" -> "%2B"   
    source = quote(columns[0]) #from
    target = quote(columns[1]) #to
    clicks = columns[3] #number of clicks

    #if the everyithing is ok; not bad charachter is in the strings, make a 
    #dictionary out of them and add the to them and add to the DataFrame
    if good_string(source) and good_string(target):
        df_clicks = df_clicks.append({"from":source, "clicks":clicks, "to":target},ignore_index=True)

###How many rows/recods do we have?

In [ ]:
print("number of records:",len(df_clicks))

##Replacing non-informative strings with null and then removing nulles

In [ ]:
#remove all rows containing empty strings of null values
df_clicks = df_clicks.replace("", np.nan)
df_clicks.dropna(inplace=True)

###How many recodes remained after removing null values?

In [27]:
#see how many rows remained:
print("number of records:",len(df_clicks))

number of records: 17834


##Merge the data of all months

In [ ]:
#summ all clicks with the identical "from" and "to" fileds. These identical rows 
#come from different months but the same source and target pages.
df_clicks = df_clicks.groupby(["from","to"]).sum() 
df_clicks.reset_index(inplace = True) #reorder the indecies after applying sum
df_clicks = df_clicks[["from","clicks","to"]] #reorder the columns

###How many recodes remained after merging months?

In [28]:
#see how many rows remained:
print("number of records:",len(df_clicks))


number of records: 17834


##Saving the results not to always need loading and preprocessing the data:

In [ ]:
#Saving the data to Google Drive
from google.colab import drive
drive.mount('/content/drive')
df_clicks.to_csv('/content/drive/wikipedia_clicks.csv')

<a name="cell-id"></a>
##If the data is already preprocessed and saved on Google Drive, just load it from the CSV file:

In [ ]:
df_clicks.from_csv('/content/drive/wikipedia_clicks.csv')

###Definition of child of a page: 
A page page_child is the child of the page_parent, if in the df_clicks dataset, there is any row in which page_parent is in the "from" and the page_child is in the "to" field.

###Rerieve page: <a name="function"></a>

In [ ]:
#from wikipedia.wikipedia import random
#Get the page related to the keyword/query
def get_page(query): 
  #Search for matching pages to the query/keyword
  results = wikipedia.search(query)
 
  if len(results) == 0:#if no page is found, as Wikipedia to suggest a page
    suggestion = wikipedia.suggest(query)
    return wikipedia.page(suggestion) #return the suggested page

  if len(results) == 1:#if just one keword is found
    try:#try to find the related page and return it
      return wikipedia.page(results)
    except:#if actually no related page is there on Wikipedia, return Null
      return None

  #we are going to find the page with the maximum number of children pages
  max_child = -1 #initial value for the maximum number of pages
  max_page = None

  for i in range(len(results)): #for the number of found keywords,
    #take keywords one by one
    res = results[i] 
    #how many children does the page have?/how many times people have gone from this page to anothe one?
    n_child = len(df_clicks[df_clicks["from"].str.contains(query,case=False)])
    if n_child > max_child: #if the number of the children is bigger than the number of the current maximum number of children
      max_child = n_child #set the new number of chilred as the maximum and 
      try: #try to find the page 
        max_page = wikipedia.page(res) 
      except:
        pass

  return max_page

##Class/Type/Abstraction/Defenition of an Article:
###Each articla is packed inside a class
<a name="class"></a>

In [1]:
#each artice and it's parameters is considered as an objet of the type class "article"
class article:
  #constructor function; initializes all attributes other than the ones which require
  #web requests using Wikipedia API.
  def __init__(self,keyword, parent = [], children = []):
    self.keyword = keyword #wikipedia keyword
    self.parent = parent #from which page we people come to the page having the "keyword"?
    self.children = children #to which page people gone from the page having the "keyword"?
    self.fill_article_info() #find URL and abstract of the page containing the "keyword"
  
  def fill_article_info(self):    
    #retrieves page title and URL and initializes the respective attributes
    page_title = get_page(self.keyword) #find the page which moslty matched the keyword
    self.page = page_title #get the title of the page
    self.article_url = self.page.url #get the urls of the page
    print(self.article_url) #print out the URL 
    print(self.page.summary) #print out the summary
    

###Article Tree Class
<a name="class_inheritance"></a>
<a name="while_loop"></a>
<a name="recursive_function"></a>

###For each search with specific depth and width, a tree data structure made of article classes is built. Instead of calling a search function recursively and saving the resulting keywords and URLs in separate lists, each found article is packed in an article class. Consecutive linked articles based on the click or search data are stored in a tree class which also preserves the relations among articles.

In [ ]:
class article_tree:

  def __init__(self, root_keyword, n_depth, n_branches, all_children=None):
    self.root_keyword = root_keyword
    self.n_depth = n_depth
    self.n_branches = n_branches
    self.all_children = {root_keyword}#all_children or set()
    self.build_tree()


  def get_children(self, keyword,n_branches):

    children = df_clicks[df_clicks["from"].str.contains(keyword,case=False)].sort_values("clicks",ascending=False).iloc[0:n_branches].to.values
    children = list(children)

    n_child = len(children)

    for ch in self.all_children:
      self.all_children.add(ch)

    if n_child < n_branches :
      results = wikipedia.search(keyword)
      i = n_child
      k = 0
      while i < n_branches and k<len(results):
        if results[k] not in self.all_children:        
          self.all_children.add(results[k])
          children.append(results[k])
          i += 1
        k += 1
    return children

  def build_children(self, root_keyword, n_depth, n_branches ): 
    
    root = article(root_keyword)
    if n_depth == 1:
      return root
    
    children_names = self.get_children(root_keyword,n_branches)
    
    children = []
    for child_name in children_names:
      
      art_ = self.build_children(child_name, n_depth-1, n_branches)

      children.append(art_)
    root.children = children
    
    return root

  def build_tree(self):  
    self.root = self.build_children(self.root_keyword, self.n_depth, self.n_branches)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
keyword = "chat gpt"
tree_1 = article_tree(keyword,3,3)